# <h1>Indexing and selecting data</h1>

<p>The axis labeling information in pandas objects serves many purposes:</p>

<ul>
<li><p>Identifies data (i.e. provides <em>metadata</em>) using known indicators,
important for analysis, visualization, and interactive console display.</p></li>
<li><p>Enables automatic and explicit data alignment.</p></li>
<li><p>Allows intuitive getting and setting of subsets of the data set.</p></li>
</ul>

<p>In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects.
The primary focus will be on Series and DataFrame as they have received more development attention in this area.</p>

<p>Note</p>

<p>The Python and NumPy indexing operators <code>[]</code> and attribute operator <code>.</code> provide quick and easy access to pandas data structures across a wide range of use cases.
This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays.
However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits.
For production code, we recommended that you take advantage of the optimized
pandas data access methods exposed in this chapter.</p>

<p>Warning</p>

<p>Whether a copy or a reference is returned for a setting operation, may
depend on the context. This is sometimes called <em>chained assignment</em> and
should be avoided. See <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-view-versus-copy">Returning a View versus Copy</a>.</p>

<p>See the <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced">MultiIndex / Advanced Indexing</a> for <code>MultiIndex</code> and more advanced indexing documentation.</p>

<p>See the <a href="https://pandas.pydata.org/docs/user_guide/cookbook.html#cookbook-selection">cookbook</a> for some advanced strategies.</p>

## <h2>Different choices for indexing</h2>

<p>Object selection has had a number of user-requested additions in order to
support more explicit location based indexing. Pandas now supports three types
of multi-axis indexing.</p>

<ul>
<li><p><code>.loc</code> is primarily label based, but may also be used with a boolean array. <code>.loc</code> will raise <code>KeyError</code> when the items are not found. Allowed inputs are:</p>

<blockquote>
<div><ul>

<li><p>A single label, e.g. <code>5</code> or <code>'a'</code> (Note that <code>5</code> is interpreted as a <em>label</em> of the index. This use is <strong>not</strong> an integer position along the index.).</p></li>

<li><p>A list or array of labels <code>['a', 'b', 'c']</code>.</p></li>

<li><p>A slice object with labels <code>'a':'f'</code> (Note that contrary to usual Python slices, <strong>both</strong> the start and the stop are included, when present in the index! See <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-slicing-with-labels">Slicing with labels</a>
and <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced-endpoints-are-inclusive">Endpoints are inclusive</a>.)</p></li>

<li><p>A boolean array (any <code>NA</code> values will be treated as <code>False</code>).</p></li>

<li><p>A <code>callable</code> function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).</p></li>

<li><p>A tuple of row (and column) indices whose elements are one of the
above inputs.</p></li>

</ul>
</div></blockquote>

<p>See more at <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-label">Selection by Label</a>.</p>
</li>

<li><p><code>.iloc</code> is primarily integer position based (from <code>0</code> to <code>length-1</code> of the axis), but may also be used with a boolean
array.
<code>.iloc</span></code> will raise <code>IndexError</code> if a requested
indexer is out-of-bounds, except <em>slice</em> indexers which allow
out-of-bounds indexing.  (this conforms with Python/NumPy <em>slice</em>
semantics).  Allowed inputs are:</p>

<blockquote>
<div><ul>

<li><p>An integer e.g. <code>5</code>.</p></li>

<li><p>A list or array of integers <code>[4,</span> <span class="pre">3,</span> <span class="pre">0]</span></code>.</p></li>

<li><p>A slice object with ints <code>1:7</span></code>.</p></li>
<li><p>A boolean array (any <code>NA</span></code> values will be treated as <code>False</code>).</p></li>

<li><p>A <code>callable</code> function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).</p></li>

<li><p>A tuple of row (and column) indices whose elements are one of the
above inputs.</p></li>

</ul>
</div></blockquote>

<p>See more at <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-integer">Selection by Position</a>, <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced">Advanced Indexing</a> and <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced-advanced-hierarchical">Advanced Hierarchical</a>.</p>
</li>

<li><p><code>.loc</code>, <code>.iloc</code>, and also <code>[]</code> indexing can accept a <code>callable</code> as indexer. See more at <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-callable">Selection By Callable</a>.</p>

<div>
<p>Note</p>

<p>Destructuring tuple keys into row (and column) indexes occurs
<em>before</em> callables are applied, so you cannot return a tuple from
a callable to index both rows and columns.</p>
</div>
</li>
</ul>

<p>Getting values from an object with multi-axes selection uses the following
notation (using <code>.loc</code> as an example, but the following applies to <code>.iloc</code> as well).
Any of the axes accessors may be the null slice <code>:</code>.
Axes left out of the specification are assumed to be <code>:</code>, e.g. <code>p.loc['a']</code> is equivalent to <code>p.loc['a', :]</code>.</p>

In [4]:
import pandas as pd
import numpy as np

In [5]:
ser = pd.Series(range(5), index=list("abcde"))

In [6]:
ser.loc[["a", "c", "e"]]

,0
a,0
c,2
e,4


In [7]:
df = pd.DataFrame(np.arange(25).reshape(5, 5), index=list("abcde"), columns=list("abcde"))

In [8]:
df.loc[["a", "c", "e"], ["b", "d"]]

,b,d
a,1,3
c,11,13
e,21,23


## <h2>Basics</h2>

<p>As mentioned when introducing the data structures in the last section, the primary function of indexing with <code>[]</code> (a.k.a. <code>__getitem__</code> for those familiar with implementing class behavior in Python) is selecting out lower-dimensional slices.
The following table shows return type values when indexing pandas objects with <code>[]</code>:</p>

<table>
<colgroup>
<col style="width: 25.0%">
<col style="width: 25.0%">
<col style="width: 50.0%">
</colgroup>
<thead>
<tr><th><p>Object Type</p></th>
<th><p>Selection</p></th>
<th><p>Return Value Type</p></th>
</tr>
</thead>
<tbody>
<tr><td><p>Series</p></td>
<td><p><code>series[label]</span></code></p></td>
<td><p>scalar value</p></td>
</tr>
<tr><td><p>DataFrame</p></td>
<td><p><code>frame[colname]</span></code></p></td>
<td><p><code>Series</span></code> corresponding to colname</p></td>
</tr>
</tbody>
</table>

<p>Here we construct a simple time series data set to use for illustrating the
indexing functionality:</p>

In [9]:
dates = pd.date_range('1/1/2000', periods=8)

In [10]:
df = pd.DataFrame(np.random.randn(8, 4), index=dates, columns=['A', 'B', 'C', 'D'])

In [11]:
df

,A,B,C,D
2000-01-01,-0.705565,-0.972453,0.128601,0.314594
2000-01-02,-0.908820,-0.392338,-1.960117,0.862061
2000-01-03,0.345560,-0.179039,0.256707,-0.051590
2000-01-04,-0.814809,1.185189,0.226401,2.395619
2000-01-05,2.042994,0.670411,-1.652306,-1.040979
2000-01-06,-0.146117,-0.406125,2.061816,-0.669343
2000-01-07,-0.630907,0.721660,0.058817,-0.073638
2000-01-08,0.622924,0.222233,0.260075,-1.244983


<p>Note</p>

<p>None of the indexing functionality is time series specific unless
specifically stated.</p>


<p>Thus, as per above, we have the most basic indexing using <code>[]</span></code>:</p>

In [12]:
s = df['A']

In [13]:
s[dates[5]]

np.float64(-0.14611729575432553)

<p>You can pass a list of columns to <code>[]</code> to select columns in that order.
If a column is not contained in the DataFrame, an exception will be
raised. Multiple columns can also be set in this manner:</p>

In [14]:
df

,A,B,C,D
2000-01-01,-0.705565,-0.972453,0.128601,0.314594
2000-01-02,-0.908820,-0.392338,-1.960117,0.862061
2000-01-03,0.345560,-0.179039,0.256707,-0.051590
2000-01-04,-0.814809,1.185189,0.226401,2.395619
2000-01-05,2.042994,0.670411,-1.652306,-1.040979
2000-01-06,-0.146117,-0.406125,2.061816,-0.669343
2000-01-07,-0.630907,0.721660,0.058817,-0.073638
2000-01-08,0.622924,0.222233,0.260075,-1.244983


In [15]:
df[['B', 'A']] = df[['A', 'B']]

In [16]:
df

,A,B,C,D
2000-01-01,-0.972453,-0.705565,0.128601,0.314594
2000-01-02,-0.392338,-0.908820,-1.960117,0.862061
2000-01-03,-0.179039,0.345560,0.256707,-0.051590
2000-01-04,1.185189,-0.814809,0.226401,2.395619
2000-01-05,0.670411,2.042994,-1.652306,-1.040979
2000-01-06,-0.406125,-0.146117,2.061816,-0.669343
2000-01-07,0.721660,-0.630907,0.058817,-0.073638
2000-01-08,0.222233,0.622924,0.260075,-1.244983


<p>You may find this useful for applying a transform (in-place) to a subset of the columns.</p>

<div class="admonition warning">
<p class="admonition-title">Warning</p>
<p>pandas aligns all AXES when setting <code class="docutils literal notranslate"><span class="pre">Series</span></code> and <code class="docutils literal notranslate"><span class="pre">DataFrame</span></code> from <code class="docutils literal notranslate"><span class="pre">.loc</span></code>.</p>
<p>This will <strong>not</strong> modify <code class="docutils literal notranslate"><span class="pre">df</span></code> because the column alignment is before value assignment.</p>

In [17]:
df[['A', 'B']]

,A,B
2000-01-01,-0.972453,-0.705565
2000-01-02,-0.392338,-0.908820
2000-01-03,-0.179039,0.345560
2000-01-04,1.185189,-0.814809
2000-01-05,0.670411,2.042994
2000-01-06,-0.406125,-0.146117
2000-01-07,0.721660,-0.630907
2000-01-08,0.222233,0.622924


In [18]:
df.loc[:, ['B', 'A']] = df[['A', 'B']]

In [19]:
df[['A', 'B']]

,A,B
2000-01-01,-0.972453,-0.705565
2000-01-02,-0.392338,-0.908820
2000-01-03,-0.179039,0.345560
2000-01-04,1.185189,-0.814809
2000-01-05,0.670411,2.042994
2000-01-06,-0.406125,-0.146117
2000-01-07,0.721660,-0.630907
2000-01-08,0.222233,0.622924


<p>The correct way to swap column values is by using raw values:</p>

In [20]:
df.loc[:, ['B', 'A']] = df[['A', 'B']].to_numpy()

In [21]:
df[['A', 'B']]

,A,B
2000-01-01,-0.705565,-0.972453
2000-01-02,-0.908820,-0.392338
2000-01-03,0.345560,-0.179039
2000-01-04,-0.814809,1.185189
2000-01-05,2.042994,0.670411
2000-01-06,-0.146117,-0.406125
2000-01-07,-0.630907,0.721660
2000-01-08,0.622924,0.222233


<p>However, pandas does not align AXES when setting <code class="docutils literal notranslate"><span class="pre">Series</span></code> and <code class="docutils literal notranslate"><span class="pre">DataFrame</span></code> from <code class="docutils literal notranslate"><span class="pre">.iloc</span></code>
because <code class="docutils literal notranslate"><span class="pre">.iloc</span></code> operates by position.</p>
<p>This will modify <code class="docutils literal notranslate"><span class="pre">df</span></code> because the column alignment is not done before value assignment.</p>

In [22]:
df[['A', 'B']]

,A,B
2000-01-01,-0.705565,-0.972453
2000-01-02,-0.908820,-0.392338
2000-01-03,0.345560,-0.179039
2000-01-04,-0.814809,1.185189
2000-01-05,2.042994,0.670411
2000-01-06,-0.146117,-0.406125
2000-01-07,-0.630907,0.721660
2000-01-08,0.622924,0.222233


In [23]:
df.iloc[:, [1, 0]] = df[['A', 'B']]

In [24]:

df[['A','B']]

,A,B
2000-01-01,-0.972453,-0.705565
2000-01-02,-0.392338,-0.908820
2000-01-03,-0.179039,0.345560
2000-01-04,1.185189,-0.814809
2000-01-05,0.670411,2.042994
2000-01-06,-0.406125,-0.146117
2000-01-07,0.721660,-0.630907
2000-01-08,0.222233,0.622924


## <h2>Attribute access</h2>

<p>You may access an index on a <code>Series</code> or  column on a <code>DataFrame</code> directly as an attribute:</p>

In [25]:
sa = pd.Series([1, 2, 3], index=list('abc'))

In [26]:
dfa = df.copy()

In [27]:
sa.b

np.int64(2)

In [28]:
dfa.A

,A
2000-01-01,-0.972453
2000-01-02,-0.392338
2000-01-03,-0.179039
2000-01-04,1.185189
2000-01-05,0.670411
2000-01-06,-0.406125
2000-01-07,0.721660
2000-01-08,0.222233


In [29]:
sa.a = 5

In [30]:
sa

,0
a,5
b,2
c,3


In [31]:
dfa['A'] = list(range(len(dfa.index)))

In [32]:
dfa

,A,B,C,D
2000-01-01,0,-0.705565,0.128601,0.314594
2000-01-02,1,-0.908820,-1.960117,0.862061
2000-01-03,2,0.345560,0.256707,-0.051590
2000-01-04,3,-0.814809,0.226401,2.395619
2000-01-05,4,2.042994,-1.652306,-1.040979
2000-01-06,5,-0.146117,2.061816,-0.669343
2000-01-07,6,-0.630907,0.058817,-0.073638
2000-01-08,7,0.622924,0.260075,-1.244983


<div class="alert alert-warning">

<p>Warning</p>

<ul>

<li><p>You can use this access only if the index element is a valid Python identifier, e.g. <code>s.1</code> is not allowed.
See <a href="https://docs.python.org/3/reference/lexical_analysis.html#identifiers">here for an explanation of valid identifiers</a>.</p></li>

<li><p>The attribute will not be available if it conflicts with an existing method name, e.g. <code>s.min</code> is not allowed, but <code>s['min']</code> is possible.</p></li>

<li><p>Similarly, the attribute will not be available if it conflicts with any of the following list: <code>index</code>,
<code>major_axis</code>, <code>minor_axis</code>, <code>items</code>.</p></li>

<li><p>In any of these cases, standard indexing will still work, e.g. <code>s['1']</code>, <code>s['min']</code>, and <code>s['index']</code> will
access the corresponding element or column.</p></li>

</ul>

</div>

<p>If you are using the IPython environment, you may also use tab-completion to see these accessible attributes.</p>

<p>You can also assign a <code>dict</code> to a row of a <code>DataFrame</code>:</p>

In [33]:
x = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})

In [34]:
x.iloc[1] = {'x': 9, 'y': 99}

In [35]:
x

,x,y
0,1,3
1,9,99
2,3,5


<p>You can use attribute access to modify an existing element of a Series or column of a DataFrame, but be careful;
if you try to use attribute access to create a new column, it creates a new attribute rather than a
new column and will this raise a <code class="docutils literal notranslate"><span class="pre">UserWarning</span></code>:</p>

In [36]:
df_new = pd.DataFrame({'one': [1., 2., 3.]})

In [37]:
df_new.two = [4, 5, 6]

/tmp/ipython-input-37-1574189772.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_new.two = [4, 5, 6]


In [38]:
df_new

,one
0,1.0
1,2.0
2,3.0


## <h2>Slicing ranges</h2>

<p>The most robust and consistent way of slicing ranges along arbitrary axes is described in the <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-integer">Selection by Position</a> section
detailing the <code>.iloc</code> method. For now, we explain the semantics of slicing using the <code>[]</span></code> operator.</p>

<p>With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels:</p>

In [39]:
s[:5]

,A
2000-01-01,-0.705565
2000-01-02,-0.908820
2000-01-03,0.345560
2000-01-04,-0.814809
2000-01-05,2.042994


In [40]:
s[::2]

,A
2000-01-01,-0.705565
2000-01-03,0.345560
2000-01-05,2.042994
2000-01-07,-0.630907


In [41]:
s[::-1]

,A
2000-01-08,0.622924
2000-01-07,-0.630907
2000-01-06,-0.146117
2000-01-05,2.042994
2000-01-04,-0.814809
2000-01-03,0.345560
2000-01-02,-0.908820
2000-01-01,-0.705565


<p>Note that setting works as well:</p>

In [42]:
s2 = s.copy()

In [43]:
s2[:5] = 0

In [44]:
s2

,A
2000-01-01,0.000000
2000-01-02,0.000000
2000-01-03,0.000000
2000-01-04,0.000000
2000-01-05,0.000000
2000-01-06,-0.146117
2000-01-07,-0.630907
2000-01-08,0.622924


<p>With DataFrame, slicing inside of <code>[]</code> <strong>slices the rows</strong>.
This is provided largely as a convenience since it is such a common operation.</p>

In [45]:
df[:3]

,A,B,C,D
2000-01-01,-0.972453,-0.705565,0.128601,0.314594
2000-01-02,-0.392338,-0.908820,-1.960117,0.862061
2000-01-03,-0.179039,0.345560,0.256707,-0.051590


In [46]:
df[::-1]

,A,B,C,D
2000-01-08,0.222233,0.622924,0.260075,-1.244983
2000-01-07,0.721660,-0.630907,0.058817,-0.073638
2000-01-06,-0.406125,-0.146117,2.061816,-0.669343
2000-01-05,0.670411,2.042994,-1.652306,-1.040979
2000-01-04,1.185189,-0.814809,0.226401,2.395619
2000-01-03,-0.179039,0.345560,0.256707,-0.051590
2000-01-02,-0.392338,-0.908820,-1.960117,0.862061
2000-01-01,-0.972453,-0.705565,0.128601,0.314594


## <h2>Selection by label</h2>

<p>Warning</p>

<p>Whether a copy or a reference is returned for a setting operation, may depend on the context.
This is sometimes called <code>chained assignment</code> and should be avoided.
See <anhref="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-view-versus-copy">Returning a View versus Copy</a>.</p>

<p>Warning</p>

<blockquote>
<div><p><code>.loc</span></code> is strict when you present slicers that are not compatible (or convertible) with the index type. For example
using integers in a <code>DatetimeIndex</code>.
These will raise a <code>TypeError</code>.</p>

In [47]:
dfl = pd.DataFrame(np.random.randn(5, 4), columns=list('ABCD'), index=pd.date_range('20130101', periods=5))

In [49]:
dfl.loc[2:3]

TypeError: cannot do slice indexing on DatetimeIndex with these indexers [2] of type int

<p>String likes in slicing <em>can</em> be convertible to the type of the index and lead to natural slicing.</p>

In [51]:
dfl.loc['20130102':'20130104']

,A,B,C,D
2013-01-02,-0.747775,-0.779910,-0.535307,0.008282
2013-01-03,-1.143510,-1.515827,-0.452967,1.531243
2013-01-04,1.550944,-1.199548,0.634772,0.339243




<p>pandas provides a suite of methods in order to have <strong>purely label based indexing</strong>. This is a strict inclusion based protocol.
Every label asked for must be in the index, or a <code>KeyError</code> will be raised.
When slicing, both the start bound <strong>AND</strong> the stop bound are <em>included</em>, if present in the index.
Integers are valid labels, but they refer to the label <strong>and not the position</strong>.</p>
<p>The <code>.loc</code> attribute is the primary access method.
The following are valid inputs:</p>

<ul>

<li><p>A single label, e.g. <code>5</code> or <code>'a'</code> (Note that <code>5</code> is interpreted as a <em>label</em> of the index.
This use is <strong>not</strong> an integer position along the index.).</p></li>

<li><p>A list or array of labels <code>['a', 'b', 'c']</code>.</p></li>

<li><p>A slice object with labels <code>'a':'f'</code> (Note that contrary to usual Python slices, <strong>both</strong> the start and the stop are included, when present in the index! See <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-slicing-with-labels">Slicing with labels</a>.</p></li>

<li><p>A boolean array.</p></li>

<li><p>A <code>callable</code>, see <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-callable">Selection By Callable</a>.</p></li>
</ul>

In [52]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))

In [53]:
s1

,0
a,1.045576
b,-0.764368
c,-0.411432
d,-1.465106
e,-0.013693
f,-0.385513


In [56]:
s1.loc['c':]

,0
c,-0.411432
d,-1.465106
e,-0.013693
f,-0.385513


In [57]:
s1.loc['b']

np.float64(-0.7643679456596952)

<p>Note that setting works as well:</p>

In [58]:
s1.loc['c'] = 0

In [59]:
s1

,0
a,1.045576
b,-0.764368
c,0.000000
d,-1.465106
e,-0.013693
f,-0.385513


<p>With a DataFrame:</p>

In [60]:
df1 = pd.DataFrame(np.random.randn(6, 4), index=list('abcdef'), columns=list('ABCD'))

In [61]:
df1

,A,B,C,D
a,-1.583071,0.607505,0.374624,1.535119
b,0.987552,0.668362,0.418253,0.175867
c,1.590869,-0.107704,-0.178175,0.942894
d,-0.683085,0.227564,1.225663,0.406755
e,-1.299408,0.273740,3.004453,0.617832
f,1.389785,-2.277182,-1.488898,0.526459


In [62]:
df1.loc[['a', 'b', 'd'], :]

,A,B,C,D
a,-1.583071,0.607505,0.374624,1.535119
b,0.987552,0.668362,0.418253,0.175867
d,-0.683085,0.227564,1.225663,0.406755


<p>Accessing via label slices:</p>

In [63]:
df1.loc['d':, 'A':'C']

,A,B,C
d,-0.683085,0.227564,1.225663
e,-1.299408,0.273740,3.004453
f,1.389785,-2.277182,-1.488898


<p>For getting a cross section using a label (equivalent to <code>df.xs('a')</code>):</p>

In [65]:
df1.loc['a']

,a
A,-1.583071
B,0.607505
C,0.374624
D,1.535119


<p>For getting values with a boolean array:</p>

In [66]:
df1.loc['a'] > 0

,a
A,False
B,True
C,True
D,True


In [67]:
df1.loc[:, df1.loc['a'] > 0]

,B,C,D
a,0.607505,0.374624,1.535119
b,0.668362,0.418253,0.175867
c,-0.107704,-0.178175,0.942894
d,0.227564,1.225663,0.406755
e,0.273740,3.004453,0.617832
f,-2.277182,-1.488898,0.526459


<p>NA values in a boolean array propagate as <code>False</code>:</p>

In [68]:
mask = pd.array([True, False, True, False, pd.NA, False], dtype="boolean")

In [69]:
mask

<BooleanArray>
[True, False, True, False, <NA>, False]
Length: 6, dtype: boolean

In [70]:
df1[mask]

,A,B,C,D
a,-1.583071,0.607505,0.374624,1.535119
c,1.590869,-0.107704,-0.178175,0.942894


<p>For getting a value explicitly:</p>

In [71]:
# this is also equivalent to ``df1.at['a','A']``
df1.loc['a', 'A']

np.float64(-1.5830708887262137)

### <h3>Slicing with labels</h3>

<p>When using <code>.loc</code> with slices, if both the start and the stop labels are present in the index, then elements <em>located</em> between the two (including them) are returned:</p>

In [73]:
s = pd.Series(list('abcde'), index=[0, 3, 2, 5, 4])

In [75]:
s.loc[3:5]

,0
3,b
2,c
5,d


<p>If at least one of the two is absent, but the index is sorted, and can be
compared against start and stop labels, then slicing will still work as
expected, by selecting labels which <em>rank</em> between the two:</p>

In [76]:
s.sort_index()

,0
0,a
2,c
3,b
4,e
5,d


In [78]:
s.sort_index().loc[1:6]

,0
2,c
3,b
4,e
5,d


<p>However, if at least one of the two is absent <em>and</em> the index is not sorted, an error will be raised (since doing otherwise would be computationally expensive, as well as potentially ambiguous for mixed type indexes). For instance, in the above example, <code>s.loc[1:6]</code> would raise <code>KeyError</code>.</p>

<p>For the rationale behind this behavior, see <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced-endpoints-are-inclusive">Endpoints are inclusive</a>.</p>

In [79]:
s = pd.Series(list('abcdef'), index=[0, 3, 2, 5, 4, 2])

In [81]:
s.loc[3:5]

,0
3,b
2,c
5,d


<p>Also, if the index has duplicate labels <em>and</em> either the start or the stop label is duplicated, an error will be raised. For instance, in the above example, <code>s.loc[2:5]</code> would raise a <code>KeyError</code>.</p>

<p>For more information about duplicate labels, see <a class="reference internal" href="https://pandas.pydata.org/docs/user_guide/duplicates.html#duplicates">Duplicate Labels</a>.</p>

## <h2>Selection by position</h2>

<p>Warning</p>

<p>Whether a copy or a reference is returned for a setting operation, may depend on the context.
This is sometimes called <code>chained </code> and should be avoided.
See <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-view-versus-copy">Returning a View versus Copy</a>.</p>

<p>Pandas provides a suite of methods in order to get <strong>purely integer based indexing</strong>. The semantics follow closely Python and NumPy slicing.
These are <code>0-based</code> indexing.
When slicing, the start bound is <em>included</em>, while the upper bound is <em>excluded</em>.
Trying to use a non-integer, even a <strong>valid</strong> label will raise an <code>IndexError</code>.</p>

<p>The <code>.iloc</code> attribute is the primary access method. The following are valid inputs:</p>

<ul>

<li><p>An integer e.g. <code>5</code>.</p></li>

<li><p>A list or array of integers <code>[4, 3, 0]</code>.</p></li>

<li><p>A slice object with ints <code>1:7</code>.</p></li>

<li><p>A boolean array.</p></li>

<li><p>A <code>callable</code>, see <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-callable">Selection By Callable</a>.</p></li>

<li><p>A tuple of row (and column) indexes, whose elements are one of the
above types.</p></li>

</ul>

In [82]:
s1 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))

In [83]:
s1

,0
0,-0.586647
2,-0.290337
4,1.303519
6,-0.102422
8,1.516520


In [84]:
s1.iloc[:3]

,0
0,-0.586647
2,-0.290337
4,1.303519


<p>Note that setting works as well:</p>

In [85]:
s1.iloc[:3] = 0

In [86]:
s1

,0
0,0.000000
2,0.000000
4,0.000000
6,-0.102422
8,1.516520


<p>With a DataFrame:</p>

In [87]:
df1 = pd.DataFrame(np.random.randn(6, 4), index=list(range(0, 12, 2)), columns=list(range(0, 8, 2)))

In [88]:
df1

,0,2,4,6
0,1.147789,0.176286,-0.258303,1.328911
2,0.364264,0.792878,0.597538,-0.105802
4,0.091214,-0.811910,-2.312617,-0.134892
6,1.168588,-0.669258,-0.431037,-0.105344
8,0.515317,0.007439,-0.313079,0.461140
10,-0.926002,1.073348,-0.995655,1.215588


<p>Select via integer slicing:</p>

In [89]:
df1.iloc[:3]

,0,2,4,6
0,1.147789,0.176286,-0.258303,1.328911
2,0.364264,0.792878,0.597538,-0.105802
4,0.091214,-0.811910,-2.312617,-0.134892


In [90]:
df1.iloc[1:5, 2:4]

,4,6
2,0.597538,-0.105802
4,-2.312617,-0.134892
6,-0.431037,-0.105344
8,-0.313079,0.461140


<p>Select via integer list:</p>

In [91]:
df.iloc[[1, 3, 5], [1, 3]]

,B,D
2000-01-02,-0.908820,0.862061
2000-01-04,-0.814809,2.395619
2000-01-06,-0.146117,-0.669343


In [92]:
df1.iloc[1:3, :]

,0,2,4,6
2,0.364264,0.792878,0.597538,-0.105802
4,0.091214,-0.811910,-2.312617,-0.134892


In [93]:
df1.iloc[1:3, :]

,0,2,4,6
2,0.364264,0.792878,0.597538,-0.105802
4,0.091214,-0.811910,-2.312617,-0.134892


In [94]:
# this is also equivalent to ``df1.iat[1,1]``
df1.iloc[1, 1]

np.float64(0.7928783687984919)

<p>For getting a cross section using an integer position (equiv to <code class="docutils literal notranslate"><span class="pre">df.xs(1)</span></code>):</p>

In [95]:
df1.iloc[1]

,2
0,0.364264
2,0.792878
4,0.597538
6,-0.105802


<p>Out of range slice indexes are handled gracefully just as in Python/NumPy.</p>

In [96]:
# these are allowed in Python/NumPy.
x = list('abcdef')

In [97]:
x

['a', 'b', 'c', 'd', 'e', 'f']

In [98]:
x[4:10]

['e', 'f']

In [99]:
x[8:10]

[]

In [101]:
s = pd.Series(x)

In [102]:
s

,0
0,a
1,b
2,c
3,d
4,e
5,f


In [103]:
s.iloc[4:10]

,0
4,e
5,f


In [104]:
s.iloc[8:10]

,0


<p>Note that using slices that go out of bounds can result in
an empty axis (e.g. an empty DataFrame being returned).</p>

In [108]:
dfl = pd.DataFrame(np.random.randn(5, 2), columns=list('AB'))

In [109]:
dfl

,A,B
0,0.653517,0.642995
1,-0.120267,0.351236
2,0.137175,-0.469661
3,-0.625725,-0.492973
4,1.850355,-1.450321


In [110]:
dfl.iloc[:, 2:3]

""
0
1
2
3
4


In [111]:
dfl.iloc[:, 1:3]

,B
0,0.642995
1,0.351236
2,-0.469661
3,-0.492973
4,-1.450321


In [113]:
dfl.iloc[4:6]

,A,B
4,1.850355,-1.450321


<p>A single indexer that is out of bounds will raise an <code class="docutils literal notranslate"><span class="pre">IndexError</span></code>.
A list of indexers where any element is out of bounds will raise an
<code class="docutils literal notranslate"><span class="pre">IndexError</span></code>.</p>

<div class="highlight-ipython notranslate"><div class="highlight"><pre id="codecell39"><span></span><span class="gp">In [100]: </span><span class="n">dfl</span><span class="o">.</span><span class="n">iloc</span><span class="p">[[</span><span class="mi">4</span><span class="p">,</span> <span class="mi">5</span><span class="p">,</span> <span class="mi">6</span><span class="p">]]</span>
<span class="gt">---------------------------------------------------------------------------</span>
<span class="ne">IndexError</span><span class="g g-Whitespace">                                </span>Traceback (most recent call last)
<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1714,</span> in <span class="ni">_iLocIndexer._get_list_axis</span><span class="nt">(self, key, axis)</span>
<span class="g g-Whitespace">   </span><span class="mi">1713</span> <span class="k">try</span><span class="p">:</span>
<span class="ne">-&gt; </span><span class="mi">1714</span>     <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">obj</span><span class="o">.</span><span class="n">_take_with_is_copy</span><span class="p">(</span><span class="n">key</span><span class="p">,</span> <span class="n">axis</span><span class="o">=</span><span class="n">axis</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">1715</span> <span class="k">except</span> <span class="ne">IndexError</span> <span class="k">as</span> <span class="n">err</span><span class="p">:</span>
<span class="g g-Whitespace">   </span><span class="mi">1716</span>     <span class="c1"># re-raise with different error message, e.g. test_getitem_ndarray_3d</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/generic.py:4172,</span> in <span class="ni">NDFrame._take_with_is_copy</span><span class="nt">(self, indices, axis)</span>
<span class="g g-Whitespace">   </span><span class="mi">4163</span><span class="w"> </span><span class="sd">"""</span>
<span class="g g-Whitespace">   </span><span class="mi">4164</span><span class="sd"> Internal version of the `take` method that sets the `_is_copy`</span>
<span class="g g-Whitespace">   </span><span class="mi">4165</span><span class="sd"> attribute to keep track of the parent dataframe (using in indexing</span>
<span class="sd">   (...)</span>
<span class="g g-Whitespace">   </span><span class="mi">4170</span><span class="sd"> See the docstring of `take` for full explanation of the parameters.</span>
<span class="g g-Whitespace">   </span><span class="mi">4171</span><span class="sd"> """</span>
<span class="ne">-&gt; </span><span class="mi">4172</span> <span class="n">result</span> <span class="o">=</span> <span class="bp">self</span><span class="o">.</span><span class="n">take</span><span class="p">(</span><span class="n">indices</span><span class="o">=</span><span class="n">indices</span><span class="p">,</span> <span class="n">axis</span><span class="o">=</span><span class="n">axis</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">4173</span> <span class="c1"># Maybe set copy if we didn't actually change the index.</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/generic.py:4152,</span> in <span class="ni">NDFrame.take</span><span class="nt">(self, indices, axis, **kwargs)</span>
<span class="g g-Whitespace">   </span><span class="mi">4148</span>     <span class="n">indices</span> <span class="o">=</span> <span class="n">np</span><span class="o">.</span><span class="n">arange</span><span class="p">(</span>
<span class="g g-Whitespace">   </span><span class="mi">4149</span>         <span class="n">indices</span><span class="o">.</span><span class="n">start</span><span class="p">,</span> <span class="n">indices</span><span class="o">.</span><span class="n">stop</span><span class="p">,</span> <span class="n">indices</span><span class="o">.</span><span class="n">step</span><span class="p">,</span> <span class="n">dtype</span><span class="o">=</span><span class="n">np</span><span class="o">.</span><span class="n">intp</span>
<span class="g g-Whitespace">   </span><span class="mi">4150</span>     <span class="p">)</span>
<span class="ne">-&gt; </span><span class="mi">4152</span> <span class="n">new_data</span> <span class="o">=</span> <span class="bp">self</span><span class="o">.</span><span class="n">_mgr</span><span class="o">.</span><span class="n">take</span><span class="p">(</span>
<span class="g g-Whitespace">   </span><span class="mi">4153</span>     <span class="n">indices</span><span class="p">,</span>
<span class="g g-Whitespace">   </span><span class="mi">4154</span>     <span class="n">axis</span><span class="o">=</span><span class="bp">self</span><span class="o">.</span><span class="n">_get_block_manager_axis</span><span class="p">(</span><span class="n">axis</span><span class="p">),</span>
<span class="g g-Whitespace">   </span><span class="mi">4155</span>     <span class="n">verify</span><span class="o">=</span><span class="kc">True</span><span class="p">,</span>
<span class="g g-Whitespace">   </span><span class="mi">4156</span> <span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">4157</span> <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">_constructor_from_mgr</span><span class="p">(</span><span class="n">new_data</span><span class="p">,</span> <span class="n">axes</span><span class="o">=</span><span class="n">new_data</span><span class="o">.</span><span class="n">axes</span><span class="p">)</span><span class="o">.</span><span class="n">__finalize__</span><span class="p">(</span>
<span class="g g-Whitespace">   </span><span class="mi">4158</span>     <span class="bp">self</span><span class="p">,</span> <span class="n">method</span><span class="o">=</span><span class="s2">"take"</span>
<span class="g g-Whitespace">   </span><span class="mi">4159</span> <span class="p">)</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/internals/managers.py:891,</span> in <span class="ni">BaseBlockManager.take</span><span class="nt">(self, indexer, axis, verify)</span>
<span class="g g-Whitespace">    </span><span class="mi">890</span> <span class="n">n</span> <span class="o">=</span> <span class="bp">self</span><span class="o">.</span><span class="n">shape</span><span class="p">[</span><span class="n">axis</span><span class="p">]</span>
<span class="ne">--&gt; </span><span class="mi">891</span> <span class="n">indexer</span> <span class="o">=</span> <span class="n">maybe_convert_indices</span><span class="p">(</span><span class="n">indexer</span><span class="p">,</span> <span class="n">n</span><span class="p">,</span> <span class="n">verify</span><span class="o">=</span><span class="n">verify</span><span class="p">)</span>
<span class="g g-Whitespace">    </span><span class="mi">893</span> <span class="n">new_labels</span> <span class="o">=</span> <span class="bp">self</span><span class="o">.</span><span class="n">axes</span><span class="p">[</span><span class="n">axis</span><span class="p">]</span><span class="o">.</span><span class="n">take</span><span class="p">(</span><span class="n">indexer</span><span class="p">)</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexers/utils.py:282,</span> in <span class="ni">maybe_convert_indices</span><span class="nt">(indices, n, verify)</span>
<span class="g g-Whitespace">    </span><span class="mi">281</span>     <span class="k">if</span> <span class="n">mask</span><span class="o">.</span><span class="n">any</span><span class="p">():</span>
<span class="ne">--&gt; </span><span class="mi">282</span>         <span class="k">raise</span> <span class="ne">IndexError</span><span class="p">(</span><span class="s2">"indices are out-of-bounds"</span><span class="p">)</span>
<span class="g g-Whitespace">    </span><span class="mi">283</span> <span class="k">return</span> <span class="n">indices</span>

<span class="ne">IndexError</span>: indices are out-of-bounds

<span class="n">The</span> <span class="n">above</span> <span class="n">exception</span> <span class="n">was</span> <span class="n">the</span> <span class="n">direct</span> <span class="n">cause</span> <span class="n">of</span> <span class="n">the</span> <span class="n">following</span> <span class="n">exception</span><span class="p">:</span>

<span class="ne">IndexError</span><span class="g g-Whitespace">                                </span>Traceback (most recent call last)
<span class="n">Cell</span> <span class="n">In</span><span class="p">[</span><span class="mi">100</span><span class="p">],</span> <span class="n">line</span> <span class="mi">1</span>
<span class="ne">----&gt; </span><span class="mi">1</span> <span class="n">dfl</span><span class="o">.</span><span class="n">iloc</span><span class="p">[[</span><span class="mi">4</span><span class="p">,</span> <span class="mi">5</span><span class="p">,</span> <span class="mi">6</span><span class="p">]]</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1191,</span> in <span class="ni">_LocationIndexer.__getitem__</span><span class="nt">(self, key)</span>
<span class="g g-Whitespace">   </span><span class="mi">1189</span> <span class="n">maybe_callable</span> <span class="o">=</span> <span class="n">com</span><span class="o">.</span><span class="n">apply_if_callable</span><span class="p">(</span><span class="n">key</span><span class="p">,</span> <span class="bp">self</span><span class="o">.</span><span class="n">obj</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">1190</span> <span class="n">maybe_callable</span> <span class="o">=</span> <span class="bp">self</span><span class="o">.</span><span class="n">_check_deprecated_callable_usage</span><span class="p">(</span><span class="n">key</span><span class="p">,</span> <span class="n">maybe_callable</span><span class="p">)</span>
<span class="ne">-&gt; </span><span class="mi">1191</span> <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">_getitem_axis</span><span class="p">(</span><span class="n">maybe_callable</span><span class="p">,</span> <span class="n">axis</span><span class="o">=</span><span class="n">axis</span><span class="p">)</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1743,</span> in <span class="ni">_iLocIndexer._getitem_axis</span><span class="nt">(self, key, axis)</span>
<span class="g g-Whitespace">   </span><span class="mi">1741</span> <span class="c1"># a list of integers</span>
<span class="g g-Whitespace">   </span><span class="mi">1742</span> <span class="k">elif</span> <span class="n">is_list_like_indexer</span><span class="p">(</span><span class="n">key</span><span class="p">):</span>
<span class="ne">-&gt; </span><span class="mi">1743</span>     <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">_get_list_axis</span><span class="p">(</span><span class="n">key</span><span class="p">,</span> <span class="n">axis</span><span class="o">=</span><span class="n">axis</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">1745</span> <span class="c1"># a single integer</span>
<span class="g g-Whitespace">   </span><span class="mi">1746</span> <span class="k">else</span><span class="p">:</span>
<span class="g g-Whitespace">   </span><span class="mi">1747</span>     <span class="n">key</span> <span class="o">=</span> <span class="n">item_from_zerodim</span><span class="p">(</span><span class="n">key</span><span class="p">)</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1717,</span> in <span class="ni">_iLocIndexer._get_list_axis</span><span class="nt">(self, key, axis)</span>
<span class="g g-Whitespace">   </span><span class="mi">1714</span>     <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">obj</span><span class="o">.</span><span class="n">_take_with_is_copy</span><span class="p">(</span><span class="n">key</span><span class="p">,</span> <span class="n">axis</span><span class="o">=</span><span class="n">axis</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">1715</span> <span class="k">except</span> <span class="ne">IndexError</span> <span class="k">as</span> <span class="n">err</span><span class="p">:</span>
<span class="g g-Whitespace">   </span><span class="mi">1716</span>     <span class="c1"># re-raise with different error message, e.g. test_getitem_ndarray_3d</span>
<span class="ne">-&gt; </span><span class="mi">1717</span>     <span class="k">raise</span> <span class="ne">IndexError</span><span class="p">(</span><span class="s2">"positional indexers are out-of-bounds"</span><span class="p">)</span> <span class="kn">from</span><span class="w"> </span><span class="nn">err</span>

<span class="ne">IndexError</span>: positional indexers are out-of-bounds
</pre>
</div>
</div>
<div class="highlight-ipython notranslate"><div class="highlight"><pre id="codecell40"><span></span><span class="gp">In [101]: </span><span class="n">dfl</span><span class="o">.</span><span class="n">iloc</span><span class="p">[:,</span> <span class="mi">4</span><span class="p">]</span>
<span class="gt">---------------------------------------------------------------------------</span>
<span class="ne">IndexError</span><span class="g g-Whitespace">                                </span>Traceback (most recent call last)
<span class="n">Cell</span> <span class="n">In</span><span class="p">[</span><span class="mi">101</span><span class="p">],</span> <span class="n">line</span> <span class="mi">1</span>
<span class="ne">----&gt; </span><span class="mi">1</span> <span class="n">dfl</span><span class="o">.</span><span class="n">iloc</span><span class="p">[:,</span> <span class="mi">4</span><span class="p">]</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1184,</span> in <span class="ni">_LocationIndexer.__getitem__</span><span class="nt">(self, key)</span>
<span class="g g-Whitespace">   </span><span class="mi">1182</span>     <span class="k">if</span> <span class="bp">self</span><span class="o">.</span><span class="n">_is_scalar_access</span><span class="p">(</span><span class="n">key</span><span class="p">):</span>
<span class="g g-Whitespace">   </span><span class="mi">1183</span>         <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">obj</span><span class="o">.</span><span class="n">_get_value</span><span class="p">(</span><span class="o">*</span><span class="n">key</span><span class="p">,</span> <span class="n">takeable</span><span class="o">=</span><span class="bp">self</span><span class="o">.</span><span class="n">_takeable</span><span class="p">)</span>
<span class="ne">-&gt; </span><span class="mi">1184</span>     <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">_getitem_tuple</span><span class="p">(</span><span class="n">key</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">1185</span> <span class="k">else</span><span class="p">:</span>
<span class="g g-Whitespace">   </span><span class="mi">1186</span>     <span class="c1"># we by definition only have the 0th axis</span>
<span class="g g-Whitespace">   </span><span class="mi">1187</span>     <span class="n">axis</span> <span class="o">=</span> <span class="bp">self</span><span class="o">.</span><span class="n">axis</span> <span class="ow">or</span> <span class="mi">0</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1690,</span> in <span class="ni">_iLocIndexer._getitem_tuple</span><span class="nt">(self, tup)</span>
<span class="g g-Whitespace">   </span><span class="mi">1689</span> <span class="k">def</span><span class="w"> </span><span class="nf">_getitem_tuple</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">tup</span><span class="p">:</span> <span class="nb">tuple</span><span class="p">):</span>
<span class="ne">-&gt; </span><span class="mi">1690</span>     <span class="n">tup</span> <span class="o">=</span> <span class="bp">self</span><span class="o">.</span><span class="n">_validate_tuple_indexer</span><span class="p">(</span><span class="n">tup</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">1691</span>     <span class="k">with</span> <span class="n">suppress</span><span class="p">(</span><span class="n">IndexingError</span><span class="p">):</span>
<span class="g g-Whitespace">   </span><span class="mi">1692</span>         <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">_getitem_lowerdim</span><span class="p">(</span><span class="n">tup</span><span class="p">)</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:966,</span> in <span class="ni">_LocationIndexer._validate_tuple_indexer</span><span class="nt">(self, key)</span>
<span class="g g-Whitespace">    </span><span class="mi">964</span> <span class="k">for</span> <span class="n">i</span><span class="p">,</span> <span class="n">k</span> <span class="ow">in</span> <span class="nb">enumerate</span><span class="p">(</span><span class="n">key</span><span class="p">):</span>
<span class="g g-Whitespace">    </span><span class="mi">965</span>     <span class="k">try</span><span class="p">:</span>
<span class="ne">--&gt; </span><span class="mi">966</span>         <span class="bp">self</span><span class="o">.</span><span class="n">_validate_key</span><span class="p">(</span><span class="n">k</span><span class="p">,</span> <span class="n">i</span><span class="p">)</span>
<span class="g g-Whitespace">    </span><span class="mi">967</span>     <span class="k">except</span> <span class="ne">ValueError</span> <span class="k">as</span> <span class="n">err</span><span class="p">:</span>
<span class="g g-Whitespace">    </span><span class="mi">968</span>         <span class="k">raise</span> <span class="ne">ValueError</span><span class="p">(</span>
<span class="g g-Whitespace">    </span><span class="mi">969</span>             <span class="s2">"Location based indexing can only have "</span>
<span class="g g-Whitespace">    </span><span class="mi">970</span>             <span class="sa">f</span><span class="s2">"[</span><span class="si">{</span><span class="bp">self</span><span class="o">.</span><span class="n">_valid_types</span><span class="si">}</span><span class="s2">] types"</span>
<span class="g g-Whitespace">    </span><span class="mi">971</span>         <span class="p">)</span> <span class="kn">from</span><span class="w"> </span><span class="nn">err</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1592,</span> in <span class="ni">_iLocIndexer._validate_key</span><span class="nt">(self, key, axis)</span>
<span class="g g-Whitespace">   </span><span class="mi">1590</span>     <span class="k">return</span>
<span class="g g-Whitespace">   </span><span class="mi">1591</span> <span class="k">elif</span> <span class="n">is_integer</span><span class="p">(</span><span class="n">key</span><span class="p">):</span>
<span class="ne">-&gt; </span><span class="mi">1592</span>     <span class="bp">self</span><span class="o">.</span><span class="n">_validate_integer</span><span class="p">(</span><span class="n">key</span><span class="p">,</span> <span class="n">axis</span><span class="p">)</span>
<span class="g g-Whitespace">   </span><span class="mi">1593</span> <span class="k">elif</span> <span class="nb">isinstance</span><span class="p">(</span><span class="n">key</span><span class="p">,</span> <span class="nb">tuple</span><span class="p">):</span>
<span class="g g-Whitespace">   </span><span class="mi">1594</span>     <span class="c1"># a tuple should already have been caught by this point</span>
<span class="g g-Whitespace">   </span><span class="mi">1595</span>     <span class="c1"># so don't treat a tuple as a valid indexer</span>
<span class="g g-Whitespace">   </span><span class="mi">1596</span>     <span class="k">raise</span> <span class="n">IndexingError</span><span class="p">(</span><span class="s2">"Too many indexers"</span><span class="p">)</span>

<span class="nn">File ~/work/pandas/pandas/pandas/core/indexing.py:1685,</span> in <span class="ni">_iLocIndexer._validate_integer</span><span class="nt">(self, key, axis)</span>
<span class="g g-Whitespace">   </span><span class="mi">1683</span> <span class="n">len_axis</span> <span class="o">=</span> <span class="nb">len</span><span class="p">(</span><span class="bp">self</span><span class="o">.</span><span class="n">obj</span><span class="o">.</span><span class="n">_get_axis</span><span class="p">(</span><span class="n">axis</span><span class="p">))</span>
<span class="g g-Whitespace">   </span><span class="mi">1684</span> <span class="k">if</span> <span class="n">key</span> <span class="o">&gt;=</span> <span class="n">len_axis</span> <span class="ow">or</span> <span class="n">key</span> <span class="o">&lt;</span> <span class="o">-</span><span class="n">len_axis</span><span class="p">:</span>
<span class="ne">-&gt; </span><span class="mi">1685</span>     <span class="k">raise</span> <span class="ne">IndexError</span><span class="p">(</span><span class="s2">"single positional indexer is out-of-bounds"</span><span class="p">)</span>

<span class="ne">IndexError</span>: single positional indexer is out-of-bounds
</pre>
</div>
</div>
</section>